# Multi-Area Network Simulation and Analysis

This Jupyter Notebook documents the workflow for simulating and analyzing a multi-area network model. The notebook is structured into several sections, each focusing on a specific part of the process, from setting up the environment to running simulations and performing analyses. Below is an overview of the key steps involved:

1. **Setting Up the Environment**: Import necessary modules and define paths and parameters. This includes importing default network, simulation, and analysis parameters, as well as setting the base path for experiments. These steps are crucial for ensuring that the subsequent steps in the workflow have access to the required configurations and data.

2. **Creating the Network**: Utilize the `NeuronNumbers` and `SynapseNumbers` classes to calculate the number of neurons and synapses in the network. The network is then created using these calculations and exported for further use.

3. **Running the Simulation**: Set up and run the simulation using the defined network and simulation parameters. The simulation results are exported and can be used for analysis.

4. **Analysis**: Perform various analyses on the simulation results, including plotting raster plots and calculating mean firing rates. The analysis results are exported for further inspection.

In [1]:
import os
import sys
import importlib.util

sys.path.append('./src/')

from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_net_params import params as net_params
from data_preprocessing.cytoarchitecture import NeuronNumbers
from data_preprocessing.connectivity import SynapseNumbers
from network import Network

In [ ]:
# Import the default simulation parameters
from default_sim_params import params as sim_params
from network import networkDictFromDump
from simulation import Simulation

In [3]:
# Import the default analysis parameters
from simulation import simulationDictFromDump
from analysis import Analysis
from default_ana_params import params as ana_params

In [4]:
# Set base path
base_path = os.getcwd()
experiments_path = os.path.join(base_path, 'experiments')

### Setting Up the Environment

In this section, we set up the environment by importing necessary modules and defining paths and parameters. This includes importing default network, simulation, and analysis parameters, as well as setting the base path for experiments. These steps are crucial for ensuring that the subsequent steps in the workflow have access to the required configurations and data.

In [5]:
# Set default parameters
outpath = net_params['outpath']

# Set simulation parameters
sim_params['t_sim'] = 2000.0
sim_params['master_seed'] = 2903

# Set plotting parameters
ana_params['plotRasterArea']['low'] = 1600
ana_params['plotRasterArea']['high'] = 2000
ana_params['plotRasterArea']['fraction'] = 0.5

# Set scaling parameters
scaling_factor = 0.01
net_params['N_scaling'] = scaling_factor
net_params['K_scaling'] = scaling_factor

# Mean fullscale firing rate
net_params['fullscale_rates'] = './simulated_data/base_theory_rates.pkl'

### Creating the network

Here the network is created using the `NeuronNumbers` and `SynapseNumbers` classes. The `NeuronNumbers` class calculates the number of neurons in the network. The `SynapseNumbers` class calculates the number of synapses in the network and the synaptic weights.

In [ ]:
# Create Network class
NN = NeuronNumbers(
    surface_area=net_params['surface_area'],
    **net_params['cytoarchitecture_params']
)

In [7]:
# Create Synapse class
SN = SynapseNumbers(
    NN=NN,
    **net_params['predictive_connectomic_params']
)

In [8]:
# Create Network
net = Network(NN, SN, net_params)

# Export the network
net.dump(outpath)

# Get the network hash
net_hash = net.getHash()

# To access net object key, use net.net.keys()
# Ex.: to access neuron numbers, use net.net['neuron_numbers']

### Running the simulation

In [ ]:
# Read network dict
net_folder = os.path.join(outpath, net_hash)
net_dict = networkDictFromDump(net_folder)

In [10]:
# Create Simulation class, export it and calculate the hash
sim = Simulation(sim_params, net_dict)
sim.dump(net_folder)
sim_hash = sim.getHash()

In [ ]:
net.net['neuron_numbers'].sum()

In [ ]:
# Set output directory according to hashes, instantiate the Network
# and run the simulation
data_path = os.path.join(outpath, net_hash, sim_hash)
num_threads = int(8)
sim.setup(data_path, num_threads)
sim.simulate()

### Analysis

In [ ]:
# Read simulation dict
sim_folder = os.path.join(outpath, net_hash, sim_hash)
sim_dict = simulationDictFromDump(sim_folder)

# Create Analysis class and export it
ana = Analysis(ana_params, net_dict, sim_dict, sim_folder, base_path)
ana.dump(sim_folder)

# Do the analysis
# ana.fullAnalysis()

ana.popGids = ana._readPopGids()
ana.spikes = ana._readSpikes()
for area in net.net['area_list']:
    ana.plotRasterArea(area)

In [ ]:
ana.meanFiringRate()